# Quantum Decision-Making for Pass Selection in Football
This notebook presents a **quantum optimisation pipeline** for analysing football passing decisions.

At each on-ball moment, the ball carrier has several **passing options**.  
The objective is to determine:

> Which pass is optimal, given the geometric context?

Each decision is converted into a small **QUBO** and solves using the **Quantum Approximate Optimization Algorithm (QAOA)**.  

---

## Optimization Formulation

Each decision moment is represented as a discrete optimization problem over candidate
passes available to the ball carrier. The problem is encoded as a Quadratic Unconstrained
Binary Optimization (QUBO):

$$
\min_{x \in \{0,1\}^n} \quad x^\top Q x
$$


where:
- \(x_i = 1\) indicates selection of the \(i\)-th pass option,
- \(Q\) encodes pairwise interactions between passes, incorporating spatial separation,
  feasibility constraints, and contextual penalties.


---

Viable passing options are identified using football-realistic constraints:

- The receiver must be a **teammate**
- The receiver must not be the **ball carrier**
- The pass must lie within a practical passing range

For each candidate pass, two geometric features are computed:

- **Pass distance**
- **Pass angle** relative to the attacking direction

Each candidate pass is assigned an interpretable value score based on two principles:

- **Forward progression:** passes aligned with the opponent’s goal are valued higher than sideways or backward options
- **Effective distance:** mid-range progressive passes are rewarded, while very short or excessively long passes are penalised

Scores are normalised and used to construct the **QUBO** objective, where selecting exactly one pass corresponds to the optimal decision.

To quantify decision quality, two lightweight metrics are used:

- **Decision Efficiency Score (DES):** measures how close the selected pass is to the QUBO-optimal pass
- **Quantum Evaluation Score (QES):** reflects the value assigned to the selected pass by the quantum solution

---

In [1]:
#Running on colab
!git clone https://github.com/niirvikk/SkillCorner-Submission.git

Cloning into 'SkillCorner-Submission'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 213 (delta 49), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (213/213), 7.88 MiB | 3.67 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [2]:
# === Install dependencies ===
!pip install -q \
    "qiskit>=0.46" \
    "qiskit-algorithms>=0.4.0" \
    qci-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.7.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
scikit-image 0.25.2 requires networkx>=3.0, but you have networkx 2.8.8 which is incompatible.
mapclassify 2.10.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.
momepy 0.10.0 requires networkx>=3.2, but you have networkx 2.8.8 which is incompatible.


In [3]:
from pathlib import Path
import sys

REPO_ROOT = Path("/content/SkillCorner-Submission").resolve()
SRC_DIR   = REPO_ROOT / "src"
DATA_DIR  = SRC_DIR / "data"

assert REPO_ROOT.exists(), REPO_ROOT
assert SRC_DIR.exists(), SRC_DIR
assert DATA_DIR.exists(), DATA_DIR

# Make src importable
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

print("Repo root:", REPO_ROOT)
print("Src dir :", SRC_DIR)
print("Data dir:", DATA_DIR)


Repo root: /content/SkillCorner-Submission
Src dir : /content/SkillCorner-Submission/src
Data dir: /content/SkillCorner-Submission/src/data


## Solution – QAOA vs Optimal vs Actual Pass

To illustrate how the quantum decision pipeline operates, a real on-ball decision taken during match play is examined.

- **Decision moment:** #2  
- **Period:** 1  
- **Event ID:** `9_10`  
- **Ball carrier (tracking ID):** `135056`  

---

### Candidate Passes

At this moment, the ball carrier has **6 viable passing options** identified by our candidate generation pipeline:

| candidate_idx | player_id | distance (m) | angle (deg) | value_score |
|--------------:|-----------|--------------|-------------|-------------|
| 0 | 14736  | 6.44  | -39.6  | 0.400 |
| 1 | 133501 | 7.08  | 47.5   | 0.354 |
| 2 | 11117  | 10.21 | -43.8  | **0.410** |
| 3 | 157294 | 11.72 | -116.2 | 0.117 |
| 4 | 38673  | 11.96 | -153.3 | 0.120 |
| 5 | 811820 | 11.97 | 91.3   | 0.120 |

- **distance** measures separation from the ball carrier to the receiver  
- **angle** captures forward progression relative to the attacking direction  
- **value_score** combines forwardness and useful distance into a scalar score in \([0,1]\)

Among these, **candidate 2** provides the highest overall value.

---

### Brute-Force Optimal Pass (Ground Truth)

The QUBO is solved **exactly** by evaluating all feasible pass assignments:

- **Optimal candidate index:** `2`  
- **Optimal value_score:** **0.410**

Thus, the optimal pass in this situation is to **player `11117` (candidate 2)**.

---

### QAOA Solution

Running QAOA on the Ising Hamiltonian derived from this decision yields:

- **QAOA candidate index:** `2`  
- **QAOA value_score:** **0.410**  
- **Matches optimal?** **Yes**

QAOA perfectly recovers the globally optimal pass.

---

### Actual On-Pitch Decision

From the event data, the pass actually played by the ball carrier was:

- **Actual candidate index:** `1`  
- **Actual value_score:** `0.354`

This is a reasonable forward option, but it is **not the highest-value pass available**.

---

### Error Metrics

The gap between the realised decision and the optimal one is quantified by:

- **Decision Error Score (DES):**  
  $$
  DES = v_{\text{optimal}} - v_{\text{actual}}
      = 0.410 - 0.354
      = 0.056
  $$

- **Quantum Error Score (QES):**  
  $$
  QES = v_{\text{QAOA}} - v_{\text{actual}}
      = 0.410 - 0.354
      = 0.056
  $$

---

### Interpretation

- The on-pitch decision is **mildly suboptimal**, giving up **0.056** units of passing value.  
- QAOA correctly identifies the most progressive option (candidate `2`) that was not chosen.  
- In this moment, **quantum optimisation would have improved the decision** relative to what occurred on the pitch.

---


In [4]:
import importlib

core = importlib.import_module("core")




Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

,n_moments,mean_des,median_des,std_des,match_rate,mean_qes,cluster,cluster_label
carrier_id,,,,,,,,
135056,3,0.231676,0.26186,0.163078,1.0,0.231676,1,Conservative / Safe


Decision moment #2
Carrier ID      : 135056
Event ID        : 9_10
Period          : 1

Candidate passes:


,candidate_idx,player_id,distance,angle,value_score
0,0,14736,6.44,-39.6,0.400
1,1,133501,7.08,47.5,0.354
2,2,11117,10.21,-43.8,0.410
3,3,157294,11.72,-116.2,0.117
4,4,38673,11.96,-153.3,0.120
5,5,811820,11.97,91.3,0.120



Brute-force optimal pass:
  optimal_candidate_idx : 2
  optimal_value_score   : 0.410

QAOA solution:
  qaoa_candidate_idx    : 2
  qaoa_value_score      : 0.410
  matches optimal?      : True

Actual on-pitch decision:
  actual_value_score    : 0.354
  inferred_actual_idx   : 1

Error metrics:
  DES (opt - actual)    : 0.056
  QES (QAOA - actual)   : 0.056

--- Summary ---
Optimal idx/value   : 2 (v = 0.410)
QAOA idx/value      : 2 (v = 0.410)   match_opt = True
Actual idx/value    : 1 (v = 0.354)
DES (opt - actual)  : 0.056
QES (QAOA - actual) : 0.056


##Conclusion

This notebook demonstrates that football passing decisions can be formulated and solved as explicit optimisation problems using quantum methods. By evaluating the quality of feasible actions directly from the spatial configuration of play, the approach avoids reliance on learned historical patterns and enables decision-level analysis. Beyond passing, the same formulation can be extended to other football decisions, such as shooting selection, ball progression, and defensive positioning, highlighting quantum optimisation as a promising direction for broader decision analytics in sport.